In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [2]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[
                                        {'label':'All Sites', 'value':'ALL'},
                                        {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                        {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                        {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                        {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                    ],
                                    value='ALL',
                                    placeholder='Select Launch Site',
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( 
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)

#Computation to callback function and return graph
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        success_count = spacex_df[spacex_df['class'] == 1].groupby('Launch Site', as_index=False).agg({'class':'count'})
        success_count = success_count.rename({'class':'Success Count'}, axis=1)
        fig = px.pie(data_frame=success_count, values='Success Count', names='Launch Site', title='Total Success Launches by Site')
        return fig
        
    else:
        #Count success and failures using number of rows from filtered dataframe
        success_count = spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['class'] == 1)].shape[0]
        failure_count = spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['class'] == 0)].shape[0]

        #Prepare dataframe for pie chart
        pie_data = pd.DataFrame({
            'Outcome':['Success','Failure'],
            'Count':[success_count, failure_count]
        })

        fig = px.pie(data_frame=pie_data, values='Count', names='Outcome', title=(f'Total Success Launches for Site {entered_site}'))
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
     Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id='payload-slider', component_property='value')
    ]
)

def get_scatter_plot(entered_site, entered_slider):
    if entered_site == 'ALL':
        scatter_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= entered_slider[0]) & (spacex_df['Payload Mass (kg)'] <= entered_slider[1])]
        fig = px.scatter(data_frame=scatter_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
        return fig
    else:
        scatter_df = spacex_df[(spacex_df['Launch Site'] == entered_site) & (spacex_df['Payload Mass (kg)'] >= entered_slider[0]) & (spacex_df['Payload Mass (kg)'] <= entered_slider[1])]
        fig = px.scatter(data_frame=scatter_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()


#Examples

In [3]:
#success_count = spacex_df[spacex_df['class'] == 1].groupby('Launch Site').agg({'class':'count'})
#success_count = success_count.rename({'class':'Success Count'}, axis=1).reset_index()
#success_count

In [4]:
#Get success count by using shape and number of rows
#success_count = spacex_df[(spacex_df['Launch Site'] == 'CCAFS LC-40') & (spacex_df['class'] == 1)]
#success_count.shape

In [5]:
#Get failure count by using shape and number of rows
#failure_count = spacex_df[(spacex_df['Launch Site'] == 'CCAFS LC-40') & (spacex_df['class'] == 0)]
#failure_count.shape

In [6]:
#Get success count by using .count() method
#success_count = spacex_df[(spacex_df['Launch Site'] == 'CCAFS LC-40') & (spacex_df['class'] == 1)]['class'].count()
#success_count